# Neo4j Python API Practice (Generates Properties by Reading CSVs)
### Overview:

Teresa will write her own python code in Jupyter Notebook where...
- She will read Version 2 CSVs from ~~the Neo4j dropbox folder~~ the [GitHub repository](https://github.com/lteresah/AMLS-GraphDatabase/).
- She will create a ~~pandas dataframe~~ python list based on the data in the csv files
- She will access a Neo4j database (local)
- She will create nodes based on the information in the ~~pandas dataframe~~ python list
- She will query the database (in parallel with checking Neo4j Bloom) to check her work.
  
### Motivation:
- Establish python code that can be used in an automated pipeline.
- Progress toward a "touch-less" graph database and data management system.

## Step 1: Load the required packages, as usual:

In [1]:
import pandas as pd
from neo4j import GraphDatabase

## Step 2: Access and Read the CSV Files

In [4]:
user_csv = pd.read_csv("https://raw.githubusercontent.com/lteresah/AMLS-GraphDatabase/main/Stage2/CSVs_Version2/Neo4j_UsersF.csv")
otci_csv = pd.read_csv("https://raw.githubusercontent.com/lteresah/AMLS-GraphDatabase/main/Stage2/CSVs_Version2/Neo4j_OTC_Ingredients.csv")
mixpre_csv = pd.read_csv("https://raw.githubusercontent.com/lteresah/AMLS-GraphDatabase/main/Stage2/CSVs_Version2/Neo4j_op_MixMakePre.csv")
mixsamp_csv = pd.read_csv("https://raw.githubusercontent.com/lteresah/AMLS-GraphDatabase/main/Stage2/CSVs_Version2/Neo4j_op_MixMakeSamp.csv")
heat_csv = pd.read_csv("https://raw.githubusercontent.com/lteresah/AMLS-GraphDatabase/main/Stage2/CSVs_Version2/Neo4j_op_Heat.csv")
rest_csv = pd.read_csv("https://raw.githubusercontent.com/lteresah/AMLS-GraphDatabase/main/Stage2/CSVs_Version2/Neo4j_op_Rest.csv")

## Step 3: Create a ~~Pandas Structure~~ Python List based on the csvs of the OPERATION nodes

OK, so advice from the internet says to **NEVER APPEND TO A PANDAS DATAFRAME** -- bad for memory and performance.

Instead, grow a python LIST, and then convert to a pandas DataFrame *if necessary*... so I shall attempt that.

Goal:
1. Create a python LIST (with columns: Operation Type, Timestamp, User, Operation ID, Data)

Steps:
1. Create a *different block of code* *for each operation csv file* which extracts info for the first four columns
2. Declare/Initiate a Python list with the first row filled in (column names)
3. Takes the rest of the unused columns and creates the last Row (data)
4. Append to the global python list.

Note: This is *not* how things will be done in final practice... automated systems will be creating the python list in real time rather than reading from a csv.  Although annoying, I have to do this step to see if I can create a code which can read and write data with differing numbers and types of properties. And even if I did have to read from csvs, they would presumably be made in a structure does doesn't require writing a function specifically for that file.

### Create Python List (log) of Operations called operation_log

In [2]:
## Function for creating python lists from Operation Node Pandas files
def make_operation_log(optype, csvfile, i = 0):
    operation_log = []
    for row in range(len(csvfile)):
        Op_Type = optype
        Timestamp = csvfile.loc[row].at[f"{optype} Timestamp"]
        User = csvfile.loc[row].at["Executor"]
        Op_ID = csvfile.loc[row].at[f"{optype} Operation ID"]
        Op_Data = {}
        for column in csvfile.columns:
            Op_Data[column] =  csvfile.loc[row].at[column]
        operation_log.append([Op_Type, Timestamp, User, Op_ID, Op_Data])
    return operation_log

In [5]:
#create all the python lists
mixpre_log = make_operation_log('Mix', mixpre_csv)
mixsamp_log = make_operation_log('Mix', mixsamp_csv)
heat_log = make_operation_log('Heat', heat_csv)
rest_log = make_operation_log('Rest', rest_csv)

In [232]:
# #Declare the empty python list:
# operation_log = [["Operation Type", "Timestamp", "User", "Operation ID", "Data"]]
# # Combine all the logs into one long python list
# for entry in mixpre_log:
#     operation_log.append(entry)
# for entry in heat_log:
#     operation_log.append(entry)
# for entry in rest_log:
#     operation_log.append(entry)
# for entry in mixsamp_log:
#     operation_log.append(entry)
# for row in operation_log:
#     print(row)

In [6]:
# Does the same as above without iterating (I THINK) and is therefore faster.  Maybe it's just the same...
# Declare the empty python list:
operation_log = [["Operation Type", "Timestamp", "User", "Operation ID", "Data"]]
unsorted_log = []
# Combine all the logs into one long python list
unsorted_log.extend(mixpre_log)
unsorted_log.extend(heat_log)
unsorted_log.extend(rest_log)
unsorted_log.extend(mixsamp_log)
sorted_log = sorted(unsorted_log,key = lambda x: x[1]) # sorting by timestamp to avoid conflicts when creating relationships
operation_log.extend(sorted_log)

## Step 4: Connect to a database:

### Create and Run a local DATABASE

Start your own local DBMS using the [Neo4j Desktop App](https://neo4j.com/download/?utm_source=Google&utm_medium=PaidSearch&utm_campaign=Evergreen&utm_content=AMS-Search-SEMBrand-Evergreen-None-SEM-SEM-NonABM&utm_term=download%20neo4j&utm_adgroup=download&gad_source=1&gclid=Cj0KCQjwpNuyBhCuARIsANJqL9Mfw2KSzysHnaaX0w_SPaPP49aDQPg5k6T-joWu_UnTcMYiWsrE4NEaAm4TEALw_wcB).

The default Bolt Port that is used should be 7687, but you can change the code to match your settings.

### Establish the DRIVER

#### Local Option

In [7]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687" # Specify URI of already running database
AUTH = ("neo4j","thisispractice") # Enter username and password (this should probably be a reqeusted input in final product)

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

## Step 5: Create the nodes and relationships

In [8]:
### This cell will probably become part of a python module one day (once I learn how to do that)

# Dictionary containing the allowed nodes in the DB, their UID row, and their functions.
DB_Nodes = {'Objects':
                [{'name': 'OTC Ingredient', 'UID': 'OTC ID', 'functions': ['item']},
                {'name': 'User', 'UID': 'kerberos','functions': ['person']},
                {'name': 'Precursor', 'UID': 'Precursor ID' , 'functions': ['item']},
                {'name': 'Sample', 'UID': 'Sample ID', 'functions': ['item']}], 
            'Operations':
                [{'name': 'Mix', 'UID': 'Mix Operation ID', 'functions': ['item_consuming','item_creating']},
                {'name': 'Heat', 'UID': 'Heat Operation ID','functions': ['item_acting']},
                {'name': 'Rest', 'UID': 'Rest Operation ID', 'functions': ['item_acting']}]
            }

# Function to get name of uid column given the node name
def get_uidname(nodename, DB_Nodes = DB_Nodes):
    found = False
    for object in DB_Nodes['Objects']:
        if object['name'] == nodename:
            uid = object['UID']
            found = True
            return uid  
    for operation in DB_Nodes['Operations']:
        if operation['name'] == nodename:
            uid = operation['UID']
            found = True
            return uid
    if not found:
        print(f"{nodename} is not registered as a Node in the DataBase.")
        return NameError
    
# Function that creates constraints based on current dictionary of allowed nodes.

# Function to get functions of node given the node name
def get_functions(nodename, DB_Nodes = DB_Nodes):
     found = False
     for object in DB_Nodes['Objects']:
        if object['name'] == nodename:
            functions = object['functions']
            found = True
            return functions
     for operation in DB_Nodes['Operations']:
        if operation['name'] == nodename:
            functions = operation['functions']
            found = True
            return functions
     if not found:
        print(f"{nodename} is not registered as a Node in the DataBase.")
        return NameError
    
    

In [23]:
### This cell will probably become part of a python module one day (once I learn how to do that)

# Function that generates cypher code for creating CONSTRAINTS based on the nodename
def gencypher_createConstraint(nodename):
    for object in DB_Nodes['Objects']:
        if object['name'] == nodename:
            uid = get_uidname(nodename)
            cypher_string = (f"""
                             CREATE CONSTRAINT `{uid} {nodename}_uniq` IF NOT EXISTS
                             FOR (n: `{nodename}`) // is full label for this type of node
                             REQUIRE (n.`{uid}`) IS UNIQUE
                             """)
    for operation in DB_Nodes['Operations']:
        if operation['name'] == nodename:
            uid = get_uidname(nodename)
            cypher_string = (f"""
                             CREATE CONSTRAINT `{uid} {nodename}_uniq` IF NOT EXISTS
                             FOR (n: `{nodename}`) // is full label for this type of node
                             REQUIRE (n.`{uid}`) IS UNIQUE
                             """)
    return cypher_string
            

# Function that generates the cypher code for OBJECT node creation, given a pandas DataFrame
def gencypher_createObjectNode(nodename, csvfile):
    uid = get_uidname(nodename)
    cypher_string = ("MERGE (n: " + f"`{nodename}`" + "{" +  f"`{uid}` : $`{uid}`" "})") # replace this with something more dynamic in future
    for property in csvfile.columns:
        next_string = f"\nSET n.`{property}` = $`{property}`" #Probably not good to do security wise... Injection risk
        cypher_string = (cypher_string + next_string)
    return cypher_string

# Function that generates the cypher code for OPERATION node creation, given a row in the python list
def gencypher_createOperationNode(row):
    nodename = row[0]
    uid = get_uidname(nodename)
    data = row[4]
    cypher_string = (f"MERGE (n:Operation:`{nodename}`" + "{" + f"`{uid}` : " + f"$`{uid}`" + "})")
    for key in data.keys():
        next_string = f"\nSET n.`{key}` = $`{key}`" #Probably not good to do security wise... Injection risk
        cypher_string = (cypher_string + next_string)
    params = {}
    params = data
    return (cypher_string, params)

# Function that generates the cypher code for RELATIONS going to/from OPERATION node, given a row in the python list (operation_log)
def gencypher_createRelations(row):
    nodename = row[0]
    uid = get_uidname(nodename)
    functions = get_functions(nodename)
    data = row[4]
    item_consuming = False
    item_creating = False
    item_acting = False
    if 'item_consuming' in functions:
        item_consuming = True
        ing_types = [value for key, value in data.items() if 'Ing' in key and 'Type' in key]
        ing_IDs = [value for key, value in data.items() if 'Ing' in key and 'ID' in key]
    if 'item_creating' in functions:
        item_creating = True
        created_types = [value for key, value in data.items() if 'New Object Type' in key]
        created_IDs = [value for key, value in data.items() if 'New Object ID' in key]
        #created_description = [value for key, value in data.items() if 'New Object Description' in key]
    if 'item_acting' in functions:
        item_acting = True
        acted_types = [value for key, value in data.items() if 'Object Type' in key]
        acted_IDs = [value for key, value in data.items() if 'Object ID' in key]
    # Create Relationships GOING TO node
    cypher_string = (f"MATCH (target: Operation:`{nodename}`" + " { " + f"`{uid}`: $`{uid}`" + " })")
    # Connecting to Executor
    user_string = ("""MATCH (user: `User` { `kerberos` : $`Executor` })
                   MERGE (user) -[:EXECUTED]-> (target)""")
    cypher_string = (cypher_string + user_string)
    # Connecting to Ingredients
    if item_consuming:
        print("\tConnecting to Ingredients")
        for i in range(len(ing_IDs)):
            ing_uid = get_uidname(ing_types[i])
            begin_string = ("\nWITH target")
            cypher_string = (cypher_string + begin_string)
            match_ing_string = (f"\nMATCH (Ing{i+1}: `{ing_types[i]}` " +  "{ " + f"`{ing_uid}` : $`Ing {i+1} ID`"  +   " })")
            cypher_string = (cypher_string + match_ing_string)
            make_rel_string = f"\nMERGE (Ing{i+1}) -[: `WENT_TO`]-> (target)"
            cypher_string = (cypher_string + make_rel_string)
    # Connecting to Created Objects
    if item_creating:
        print("\tCreating/Connecting to Creations")
        for i in range(len(created_IDs)):
            created_uid = get_uidname(created_types[i])
            begin_string = ("\nWITH target")
            cypher_string = (cypher_string + begin_string)
            makeproduct_string = (f"\nMERGE (Created{i+1}: `{created_types[i]}` " +  "{ " + f"`{created_uid}` : $`New Object ID`"  +   " })")
            cypher_string = (cypher_string + makeproduct_string)
            make_rel_string = f"\nMERGE (Created{i+1}) <-[: `CREATED`]- (target)"
            cypher_string = (cypher_string + make_rel_string)
    # Connecting to Acted Objects
    if item_acting:
        print("\tConnecting to acted Object(s)")
        for i in range(len(acted_IDs)):
            acted_uid = get_uidname(acted_types[i])
            pass_string = ("\nWITH target")
            cypher_string = (cypher_string + pass_string)
            match_object_string = (f"\nMATCH (Acted{i+1}: `{acted_types[i]}` " +  "{ " + f"`{acted_uid}` : $`{nodename} Object ID`"  +   " })")
            cypher_string = (cypher_string + match_object_string)
            make_rel_string = f"\nMERGE (Acted{i+1}) -[: `WENT_TO`]-> (target)"
            cypher_string = (cypher_string + make_rel_string)
            # For item acting, updates the event stream tracking relationship -[:THEN]-> after creating the [:WENT_TO] relationship
            # This is really really hard to do for timestamps that have different names.  Lesson learned.
            add_eventstream_string = (f"""\n
                                        WITH Acted{i+1}
                                        MATCH (Acted{i+1})-[r:WENT_TO]->(op:Operation)
                                        WITH Acted{i+1}, op, properties(op) as properties, [key in keys(op) where key contains 'Timestamp'] as timestamp
                                        WITH Acted{i+1}, op ORDER BY properties[timestamp[0]]
                                        WITH Acted{i+1}, collect(op) as Operations
                                        FOREACH (j IN RANGE(0,size(Operations)-2)|
                                            FOREACH (before in [Operations[j]]|
                                                FOREACH(after in [Operations[j+1]]|
                                                    MERGE (before)-[:THEN]->(after))))""")
        cypher_string = (cypher_string + add_eventstream_string)
    # Setting the paramaters        
    params = {}
    params = data
    return (cypher_string, params)


In [20]:
# Goes through dictionary of nodes in the DB and creates constraints for each node
def db_setConstraints(DB_Nodes = DB_Nodes):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            for object in DB_Nodes['Objects']:
                nodename = object['name']
                cypherstring_constraint = gencypher_createConstraint(nodename)
                session.run(cypherstring_constraint)
            for operation in DB_Nodes['Operations']:
                nodename = operation['name']
                cypherstring_constraint = gencypher_createConstraint(nodename)
                session.run(cypherstring_constraint)


# Function that reads csv file for OBJECT type nodes and then creates the nodes
def db_pushObjects(nodename, csvfile):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            cypherstring_nodes = gencypher_createObjectNode(nodename, csvfile)
            csv_dict = csvfile.to_dict(orient = 'records') #changing the pd dataframe to a dict for iteration through rows.
            for row in csv_dict:
                params = {}
                params = row
                session.run(cypherstring_nodes, params)


# Function that creates OPERATION nodes+relationships based on the entries in the operation_log (python list)
def db_pushOperations(operation_log, count = 0):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            for entry in operation_log[1:]: # Iterate over log entries -- skip the first row which is just column names.
               print(f"Pushing entry[{count}] to DB for operation type: {entry[0]}:")
               cyphernode_string, cyphernode_params = gencypher_createOperationNode(entry)
               session.run(cyphernode_string, cyphernode_params)
               print(f"\tCompleted node creation")
               cypherrel_string, cypherrel_params = gencypher_createRelations(entry)
               session.run(cypherrel_string, cypherrel_params)
               print(f"\tCompleted connections")
               count = count + 1

In [ ]:
# Set Constraints
db_setConstraints()
# Push Objects
db_pushObjects('User', user_csv)
db_pushObjects('OTC Ingredient', otci_csv)
# Push Operations
db_pushOperations(operation_log)